In [1]:
import csv,sys
from collections import namedtuple
import numpy as np
import os
from PIL import Image
from sklearn.feature_extraction import image
from sklearn.feature_extraction.image import extract_patches
#from blockwise_view import blockwise_view
from skimage.util.shape import view_as_windows
import csv
from collections import namedtuple
import random

In [2]:
import pandas as pd
pd.set_option('display.max_rows', 50000)
pd.set_option('display.max_columns', 50000)
pd.set_option('display.width', 100000)
#pd.set_option('display.max_colwidth', -1)
np.set_printoptions(threshold=sys.maxsize)

In [3]:
def blockwise_view(a, blockshape, aslist=False, require_aligned_blocks=True):
    """
    Return a 2N-D view of the given N-D array, rearranged so each ND block (tile)
    of the original array is indexed by its block address using the first N
    indexes of the output array.

    Note: This function is nearly identical to ``skimage.util.view_as_blocks()``, except:
          - "imperfect" block shapes are permitted (via require_aligned_blocks=False)
          - only contiguous arrays are accepted.  (This function will NOT silently copy your array.)
            As a result, the return value is *always* a view of the input.

    Args:
        a: The ND array

        blockshape: The tile shape

        aslist: If True, return all blocks as a list of ND blocks
                instead of a 2D array indexed by ND block coordinate.

        require_aligned_blocks: If True, check to make sure no data is "left over"
                                in each row/column/etc. of the output view.
                                That is, the blockshape must divide evenly into the full array shape.
                                If False, "leftover" items that cannot be made into complete blocks
                                will be discarded from the output view.

    Here's a 2D example (this function also works for ND):

    >>> a = np.arange(1,21).reshape(4,5)
    >>> print a
    [[ 1  2  3  4  5]
     [ 6  7  8  9 10]
     [11 12 13 14 15]
     [16 17 18 19 20]]

    >>> view = blockwise_view(a, (2,2), False)
    >>> print view
    [[[[ 1  2]
       [ 6  7]]
    <BLANKLINE>
      [[ 3  4]
       [ 8  9]]]
    <BLANKLINE>
    <BLANKLINE>
     [[[11 12]
       [16 17]]
    <BLANKLINE>
      [[13 14]
       [18 19]]]]

    Inspired by the 2D example shown here: http://stackoverflow.com/a/8070716/162094
    """
    assert a.flags['C_CONTIGUOUS'], "This function relies on the memory layout of the array."
    blockshape = tuple(blockshape)
    outershape = tuple(np.array(a.shape) // blockshape)
    view_shape = outershape + blockshape

    if require_aligned_blocks:
        assert (np.mod(a.shape, blockshape) == 0).all(), \
            "blockshape {} must divide evenly into array shape {}" \
                .format(blockshape, a.shape)

    # inner strides: strides within each block (same as original array)
    intra_block_strides = a.strides

    # outer strides: strides from one block to another
    inter_block_strides = tuple(a.strides * np.array(blockshape))

    # This is where the magic happens.
    # Generate a view with our new strides (outer+inner).
    view = np.lib.stride_tricks.as_strided(a,
                                              shape=view_shape,
                                              strides=(inter_block_strides + intra_block_strides))

    return view

In [4]:
result = []
masks = []
img = []
idx = []
hw = []

original_images = []
headers = []

In [5]:
patch_size = 64;
data = 'casia'+str(patch_size)
base_folder="G:/sourav/prog/notebook/Image_manipulation_detection-master/data/casia-dataset/casia_test"

In [6]:
original_images = [os.path.join(base_folder, 'probe')+'/'+x for x in os.listdir(
    os.path.join(base_folder, 'probe')) if x.endswith(".png")]
print(len(original_images))

869


In [13]:
train_rows = []
val_rows = []
test_rows = []

for v in original_images:
    rr = random.random()
    if rr<= 0.65:
        train_rows.append(v)
    elif rr <=0.75:
        val_rows.append(v)
    else:
        test_rows.append(v)
        
print(len(train_rows))
print(len(val_rows))
print(len(test_rows))

597
78
194


In [14]:
dic = {
    'train':train_rows,
    'val':val_rows,
    'test':test_rows
}

In [15]:
for str_type in ['train','val','test']:

    count = 0
    lst = []
    
    # mkdirs
    os.makedirs(os.path.join(base_folder,data,'images',str_type))
    os.makedirs(os.path.join(base_folder,data,'masks',str_type)) 

    print("str_type ",str_type)
    for img_path in dic[str_type]: # each image in dic

        IMG_PATH = img_path
        MASK_PATH = img_path.replace(
            'probe', 'mask')
        print("IMG_PATH: ",IMG_PATH)
        print("MASK_PATH: ",MASK_PATH)

        prob = Image.open(IMG_PATH)
        rgbmask = Image.open(MASK_PATH) 
        
        assert prob.size == rgbmask.size
        width,height = prob.size

        rgbmask = np.array(rgbmask)
        prob = np.array(prob)
        
        #idx = np.argmax(rgbmask,2)

        mask = np.zeros([rgbmask.shape[0],rgbmask.shape[1]])
        #mask[idx == 0] = 255 # rgb
        mask[np.where(rgbmask == 0)] = 255 # rgb

        img_patches = blockwise_view(prob,(patch_size,patch_size,3),require_aligned_blocks=False)
        mask_patches = blockwise_view(mask,(patch_size,patch_size),require_aligned_blocks=False)

        img_patches = img_patches.squeeze(axis=2)


        for h in range(img_patches.shape[0]):
            for w in range(img_patches.shape[1]):
               
                im = Image.fromarray(img_patches[h][w])
                mk = Image.fromarray(mask_patches[h][w]).convert('RGB')

                PATCH_PATH = os.path.join(
                    data, 'images', str_type, str(count)+'.png')
                MASK_PATCH_PATH = os.path.join(
                    data, 'masks', str_type, str(count)+'.png')

                im.save(base_folder+'/'+PATCH_PATH)
                mk.save(base_folder+'/'+MASK_PATCH_PATH)

                lst.append(",".join([PATCH_PATH, MASK_PATCH_PATH, MASK_PATH.replace(
                    base_folder, ''), IMG_PATH.replace(base_folder, '')]))

                count = count + 1

        print('['+str(count)+']') 

    with open(os.path.join(base_folder,data,str_type+'.txt'),'w') as f:
        f.write("\n".join(lst))    

str_type  train
IMG_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/data/casia-dataset/casia_test\probe/Sp_D_CND_A_pla0005_pla0023_0281.png
MASK_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/data/casia-dataset/casia_test\mask/Sp_D_CND_A_pla0005_pla0023_0281.png
[24]
IMG_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/data/casia-dataset/casia_test\probe/Sp_D_CNN_A_ani0049_ani0084_0266.png
MASK_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/data/casia-dataset/casia_test\mask/Sp_D_CNN_A_ani0049_ani0084_0266.png
[48]
IMG_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/data/casia-dataset/casia_test\probe/Sp_D_CNN_A_ani0053_ani0054_0267.png
MASK_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/data/casia-dataset/casia_test\mask/Sp_D_CNN_A_ani0053_ani0054_0267.png
[72]
IMG_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/data/casia-dataset/casia_test\probe/Sp_D

[744]
IMG_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/data/casia-dataset/casia_test\probe/Sp_D_CRN_A_nat0094_art0096_0389.png
MASK_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/data/casia-dataset/casia_test\mask/Sp_D_CRN_A_nat0094_art0096_0389.png
[768]
IMG_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/data/casia-dataset/casia_test\probe/Sp_D_CRN_A_nat0096_art0028_0390.png
MASK_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/data/casia-dataset/casia_test\mask/Sp_D_CRN_A_nat0096_art0028_0390.png
[792]
IMG_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/data/casia-dataset/casia_test\probe/Sp_D_CRN_A_pla0006_pla0009_0391.png
MASK_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/data/casia-dataset/casia_test\mask/Sp_D_CRN_A_pla0006_pla0009_0391.png
[816]
IMG_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/data/casia-dataset/casia_test\probe/Sp_D_CRN_A_

[1458]
IMG_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/data/casia-dataset/casia_test\probe/Sp_D_NND_A_nat0023_nat0013_0608.png
MASK_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/data/casia-dataset/casia_test\mask/Sp_D_NND_A_nat0023_nat0013_0608.png
[1482]
IMG_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/data/casia-dataset/casia_test\probe/Sp_D_NND_A_nat0031_nat0033_0609.png
MASK_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/data/casia-dataset/casia_test\mask/Sp_D_NND_A_nat0031_nat0033_0609.png
[1506]
IMG_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/data/casia-dataset/casia_test\probe/Sp_D_NND_A_nat0031_nat0096_0610.png
MASK_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/data/casia-dataset/casia_test\mask/Sp_D_NND_A_nat0031_nat0096_0610.png
[1530]
IMG_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/data/casia-dataset/casia_test\probe/Sp_D_NN

[2130]
IMG_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/data/casia-dataset/casia_test\probe/Sp_D_NNN_A_art0033_cha0095_0296.png
MASK_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/data/casia-dataset/casia_test\mask/Sp_D_NNN_A_art0033_cha0095_0296.png
[2154]
IMG_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/data/casia-dataset/casia_test\probe/Sp_D_NNN_A_art0035_cha0045_0297.png
MASK_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/data/casia-dataset/casia_test\mask/Sp_D_NNN_A_art0035_cha0045_0297.png
[2178]
IMG_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/data/casia-dataset/casia_test\probe/Sp_D_NNN_A_art0043_cha0047_0299.png
MASK_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/data/casia-dataset/casia_test\mask/Sp_D_NNN_A_art0043_cha0047_0299.png
[2202]
IMG_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/data/casia-dataset/casia_test\probe/Sp_D_NN

[2802]
IMG_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/data/casia-dataset/casia_test\probe/Sp_D_NNN_A_nat0069_cha0090_0326.png
MASK_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/data/casia-dataset/casia_test\mask/Sp_D_NNN_A_nat0069_cha0090_0326.png
[2826]
IMG_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/data/casia-dataset/casia_test\probe/Sp_D_NNN_A_nat0076_cha0067_0328.png
MASK_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/data/casia-dataset/casia_test\mask/Sp_D_NNN_A_nat0076_cha0067_0328.png
[2850]
IMG_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/data/casia-dataset/casia_test\probe/Sp_D_NNN_A_nat0077_cha0068_0329.png
MASK_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/data/casia-dataset/casia_test\mask/Sp_D_NNN_A_nat0077_cha0068_0329.png
[2874]
IMG_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/data/casia-dataset/casia_test\probe/Sp_D_NN

...

[1632]
IMG_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/data/casia-dataset/casia_test\probe/Sp_D_NRN_A_art0071_art0029_0478.png
MASK_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/data/casia-dataset/casia_test\mask/Sp_D_NRN_A_art0071_art0029_0478.png
[1656]
IMG_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/data/casia-dataset/casia_test\probe/Sp_D_NRN_A_art0077_arc0013_0464.png
MASK_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/data/casia-dataset/casia_test\mask/Sp_D_NRN_A_art0077_arc0013_0464.png
[1680]
IMG_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/data/casia-dataset/casia_test\probe/Sp_D_NRN_A_art0078_art0077_0176.png
MASK_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/data/casia-dataset/casia_test\mask/Sp_D_NRN_A_art0078_art0077_0176.png
[1704]
IMG_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/data/casia-dataset/casia_test\probe/Sp_D_NR

[2304]
IMG_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/data/casia-dataset/casia_test\probe/Sp_S_CRN_A_pla0071_pla0071_0079.png
MASK_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/data/casia-dataset/casia_test\mask/Sp_S_CRN_A_pla0071_pla0071_0079.png
[2328]
IMG_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/data/casia-dataset/casia_test\probe/Sp_S_CRN_A_pla0073_pla0073_0231.png
MASK_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/data/casia-dataset/casia_test\mask/Sp_S_CRN_A_pla0073_pla0073_0231.png
[2352]
IMG_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/data/casia-dataset/casia_test\probe/Sp_S_NND_A_art0026_art0026_0091.png
MASK_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/data/casia-dataset/casia_test\mask/Sp_S_NND_A_art0026_art0026_0091.png
[2376]
IMG_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/data/casia-dataset/casia_test\probe/Sp_S_NN

[2976]
IMG_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/data/casia-dataset/casia_test\probe/Sp_S_NNN_C_txt0067_txt0067_0067.png
MASK_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/data/casia-dataset/casia_test\mask/Sp_S_NNN_C_txt0067_txt0067_0067.png
[3000]
IMG_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/data/casia-dataset/casia_test\probe/Sp_S_NNN_C_txt0068_txt0068_0068.png
MASK_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/data/casia-dataset/casia_test\mask/Sp_S_NNN_C_txt0068_txt0068_0068.png
[3024]
IMG_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/data/casia-dataset/casia_test\probe/Sp_S_NNN_C_txt0069_txt0069_0069.png
MASK_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/data/casia-dataset/casia_test\mask/Sp_S_NNN_C_txt0069_txt0069_0069.png
[3048]
IMG_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/data/casia-dataset/casia_test\probe/Sp_S_NN

[3648]
IMG_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/data/casia-dataset/casia_test\probe/Sp_S_NNN_R_txt0030_txt0030_0030.png
MASK_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/data/casia-dataset/casia_test\mask/Sp_S_NNN_R_txt0030_txt0030_0030.png
[3672]
IMG_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/data/casia-dataset/casia_test\probe/Sp_S_NNN_R_txt0031_txt0031_0031.png
MASK_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/data/casia-dataset/casia_test\mask/Sp_S_NNN_R_txt0031_txt0031_0031.png
[3696]
IMG_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/data/casia-dataset/casia_test\probe/Sp_S_NNN_R_txt0046_txt0046_0046.png
MASK_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/data/casia-dataset/casia_test\mask/Sp_S_NNN_R_txt0046_txt0046_0046.png
[3720]
IMG_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/data/casia-dataset/casia_test\probe/Sp_S_NN

[4320]
IMG_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/data/casia-dataset/casia_test\probe/Sp_S_NNN_T_txt0059_txt0059_0059.png
MASK_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/data/casia-dataset/casia_test\mask/Sp_S_NNN_T_txt0059_txt0059_0059.png
[4344]
IMG_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/data/casia-dataset/casia_test\probe/Sp_S_NNN_T_txt0065_txt0065_0065.png
MASK_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/data/casia-dataset/casia_test\mask/Sp_S_NNN_T_txt0065_txt0065_0065.png
[4368]
IMG_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/data/casia-dataset/casia_test\probe/Sp_S_NNN_T_txt0073_txt0073_0073.png
MASK_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/data/casia-dataset/casia_test\mask/Sp_S_NNN_T_txt0073_txt0073_0073.png
[4392]
IMG_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/data/casia-dataset/casia_test\probe/Sp_S_NN